# 8-1 k-fold cross-validation

## Split data

In [1]:
import polars as pl

In [2]:
pl_production = pl.read_parquet("../../data/production.parquet")
pl_production.head()

type,length,thickness,fault_flg
str,f64,f64,bool
"""E""",274.027383,40.241131,false
"""D""",86.319269,16.906715,false
"""E""",123.940388,1.018462,false
"""B""",175.554886,16.414924,false
"""B""",244.93474,29.061081,false


In [3]:
from sklearn.model_selection import train_test_split, KFold

# hold out
train_data, test_data = train_test_split(
    pl_production,
    test_size=0.2,
    shuffle=True,
    random_state=71,
)

# F-fold cross validation
fold = KFold(
    n_splits=10,
    shuffle=True,
    random_state=71,
)


# xgboost
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

# training and validation
for train_cv_no, test_cv_no in fold.split(train_data):
    train_cv = train_data[train_cv_no]
    bst.fit(train_cv.select(["length", "thickness"]), train_cv.select("fault_flg"))
    test_cv  = train_data[test_cv_no]
    print(bst.score(test_cv.select(["length", "thickness"]), test_cv.select("fault_flg")))


0.9875
0.8875
0.95
0.925
0.9625
0.975
0.8875
0.975
0.95
0.95
